In [5]:
import pandas as pd
import geopandas as gpd
from srai.regionalizers import geocode_to_region_gdf
from srai.plotting.folium_wrapper import plot_regions
import osmnx as ox
import sys
sys.path.append('../.') 
from src.graph.create_osmnx_graph import OSMnxGraph

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
df_accidents = pd.read_csv("../data/wypadki-pl/accidents.csv")
df_accidents.drop(df_accidents[(df_accidents['mie_nazwa'] != 'Warszawa')].index, inplace=True)
df_accidents.drop(columns='uczestnicy', inplace=True)
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_accidents['wsp_gps_x'], df_accidents['wsp_gps_y'])]
gdf_acc = gpd.GeoDataFrame(df_accidents, geometry=geometry)
gdf_acc.drop(columns=['wsp_gps_x', 'wsp_gps_y'], inplace=True)
G =ox.graph.graph_from_place("Warsaw, Poland", network_type="drive")
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
gdf_nodes

,y,x,street_count,highway,ref,geometry
osmid,,,,,,
26063726,52.152787,21.017454,5,NaN,NaN,POINT (21.01745 52.15279)
26063848,52.157570,20.991392,4,NaN,NaN,POINT (20.99139 52.15757)
26063857,52.166411,20.992301,3,NaN,NaN,POINT (20.99230 52.16641)
26063923,52.170803,20.992247,3,NaN,NaN,POINT (20.99225 52.17080)
26083887,52.176026,20.995141,3,NaN,NaN,POINT (20.99514 52.17603)
...,...,...,...,...,...,...
11707394732,52.294628,21.022652,1,NaN,NaN,POINT (21.02265 52.29463)
11711719556,52.297432,21.043383,3,NaN,NaN,POINT (21.04338 52.29743)
11721637528,52.297728,21.046037,3,NaN,NaN,POINT (21.04604 52.29773)


In [24]:
graph_embedder = OSMnxGraph(gdf_acc, gdf_nodes, gdf_edges)

graph_data = graph_embedder.create_graph(aggregation_type='node')
graph_data

Data(x=[38704, 8], edge_index=[2, 44148], y=[19352], street_count=[19352], highway=[19352], ref=[19352], accidents_count=[19352], osmid=[44148], oneway=[44148], lanes=[44148], name=[44148], edge_highway=[44148], reversed=[44148], length=[44148], maxspeed=[44148], geometry=[44148], access=[44148], junction=[44148], bridge=[44148], tunnel=[44148], edge_ref=[44148], width=[44148], crs=epsg:4326)

In [25]:
graph_embedder.show_statistics()

{'Nodes': 38704,
 'Edges': 44148,
 'Nodes dim': 8,
 'Nodes class': 2,
 'Directed': True,
 'Graph density [%]': 0.003}

In [26]:
graph_data.num_features

8

In [27]:
len(graph_data.y.unique())

2

In [28]:
from src.baseline_models.GCN.supervised_node_classification import SupervisedNodeClassificationGNN
from src.baseline_models.GCN.gcn import GCNModel
import torch

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_features = graph_data.num_features
hidden_dim = 256
out_dim = 128
num_classes = len(graph_data.y.unique())

gnn = GCNModel(in_dim=num_features, hidden_dim=hidden_dim, out_dim=out_dim)

model = SupervisedNodeClassificationGNN(gnn=gnn, emb_dim=out_dim, num_classes=num_classes, lr=0.0001)

In [30]:
graph_data.x

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [31]:
graph_data

Data(x=[38704, 8], edge_index=[2, 44148], y=[19352], street_count=[19352], highway=[19352], ref=[19352], accidents_count=[19352], osmid=[44148], oneway=[44148], lanes=[44148], name=[44148], edge_highway=[44148], reversed=[44148], length=[44148], maxspeed=[44148], geometry=[44148], access=[44148], junction=[44148], bridge=[44148], tunnel=[44148], edge_ref=[44148], width=[44148], crs=epsg:4326)